# O Problema
Sliding Puzzle - Bloco Deslizante

In [1]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

In [57]:
import numpy as np

class Puzzle:
  def __init__(self, board=None, min_moves=5, max_moves=20):  
    if board is None:
      self.board = np.array([[1,2,3],[4,5,6],[7,8,-1]])
      self.empty = (2,2)

      self.shuffle(min_moves, max_moves)
    else:
      self.board = board
      e = np.where(board == -1)
      self.empty = (e[0][0], e[1][0])


  def encode(self):
    return np.array2string(self.board.flatten(), separator='')
        
  def print_board(self):
    print(self.board)
        
  def get_allowed_moves(self):
    allowed_moves = []
    
    allowed_moves += self.get_vertical_moves()
    allowed_moves += self.get_horizontal_moves()

    return allowed_moves

    
  def get_vertical_moves(self):
    allowed_moves = []
    if self.empty[0] > 0:
      allowed_moves.append((self.empty[0] - 1, self.empty[1])) # down
    if self.empty[0] < len(self.board) - 1:
      allowed_moves.append((self.empty[0] + 1, self.empty[1])) # up
        
    return allowed_moves
          
  def get_horizontal_moves(self):
    allowed_moves = []
    if self.empty[1] > 0:
      allowed_moves.append((self.empty[0], self.empty[1] - 1)) # right
    if self.empty[1] < len(self.board) - 1:
      allowed_moves.append((self.empty[0], self.empty[1] + 1)) # left
          
    return allowed_moves
  

    
  def move(self, piece_to_move):
    self.board[self.empty] = self.board[piece_to_move]
    self.board[piece_to_move] = -1
      
    self.empty = piece_to_move
        
    
    
  def check_puzzle(self):
    if np.array_equal(self.board, np.array([[1,2,3],[4,5,6],[7,8,-1]])): 
      return True
    
    return False


                
                
  def shuffle(self, min_moves, max_moves):
    rng = np.random.default_rng()
    num_of_moves = rng.integers(min_moves, max_moves)

    for i in range(num_of_moves):
      allowed_moves = self.get_allowed_moves()
      move = rng.integers(0, len(allowed_moves))

      self.move((allowed_moves[move]))


In [58]:
class Tree:
  def __init__(self, movement, puzzle, parent=None):
    self.movement = movement
    self.puzzle = puzzle
    self.children = puzzle.get_allowed_moves()
    self.parent = parent

    self.visited = False

  def create_child(self, i): # cria o filho i (adiciona o puzzle, executa o movimento, cria a árvore)
    p = Puzzle(self.puzzle.board.copy())
    p.move(self.children[i])

    self.children[i] = Tree(self.children[i], p, self)

# Resolver o quebra-cabeças usando Buscas

In [59]:
from collections import deque

In [60]:
def create_path(node):
  q = deque()
  q.append(node)

  while node.parent is not None:
    node = node.parent
    q.appendleft(node)

  return q

## Busca em largura

In [61]:
def bf_search(tree):
  q = deque()
  q.append(tree)

  s = set()

  while q: # while q is not empty
    node = q.popleft()

    enc = node.puzzle.encode()
    if enc in s:
      continue
    
    s.add(enc)

    for i, c in enumerate(node.children):
      node.create_child(i)

      q.append(node.children[i])

      if node.children[i].puzzle.check_puzzle():
        return node.children[i], create_path(node.children[i])

In [62]:
p = Puzzle(min_moves=100, max_moves=150)
t = Tree((-1,-1), p)

p.print_board()

n, path = bf_search(t)

[[ 1  3  7]
 [ 8  5  4]
 [ 2 -1  6]]


In [63]:
print("Tamanho da solução: ", len(path))

for p in path:
  print("\nPeça movida: ", p.movement)
  p.puzzle.print_board()

Tamanho da solução:  24

Peça movida:  (-1, -1)
[[ 1  3  7]
 [ 8  5  4]
 [ 2 -1  6]]

Peça movida:  (2, 0)
[[ 1  3  7]
 [ 8  5  4]
 [-1  2  6]]

Peça movida:  (1, 0)
[[ 1  3  7]
 [-1  5  4]
 [ 8  2  6]]

Peça movida:  (1, 1)
[[ 1  3  7]
 [ 5 -1  4]
 [ 8  2  6]]

Peça movida:  (2, 1)
[[ 1  3  7]
 [ 5  2  4]
 [ 8 -1  6]]

Peça movida:  (2, 2)
[[ 1  3  7]
 [ 5  2  4]
 [ 8  6 -1]]

Peça movida:  (1, 2)
[[ 1  3  7]
 [ 5  2 -1]
 [ 8  6  4]]

Peça movida:  (0, 2)
[[ 1  3 -1]
 [ 5  2  7]
 [ 8  6  4]]

Peça movida:  (0, 1)
[[ 1 -1  3]
 [ 5  2  7]
 [ 8  6  4]]

Peça movida:  (1, 1)
[[ 1  2  3]
 [ 5 -1  7]
 [ 8  6  4]]

Peça movida:  (1, 2)
[[ 1  2  3]
 [ 5  7 -1]
 [ 8  6  4]]

Peça movida:  (2, 2)
[[ 1  2  3]
 [ 5  7  4]
 [ 8  6 -1]]

Peça movida:  (2, 1)
[[ 1  2  3]
 [ 5  7  4]
 [ 8 -1  6]]

Peça movida:  (2, 0)
[[ 1  2  3]
 [ 5  7  4]
 [-1  8  6]]

Peça movida:  (1, 0)
[[ 1  2  3]
 [-1  7  4]
 [ 5  8  6]]

Peça movida:  (1, 1)
[[ 1  2  3]
 [ 7 -1  4]
 [ 5  8  6]]

Peça movida:  (1, 2)
[[ 1  2 

## Busca em Profundidade

In [64]:
def df_search(node):
  q = deque()
  s = set()

  node.visited = True
  q.append(node)

  s.add(node.puzzle.encode())

  while q:
    node = q[-1]

    for i in range(len(node.children)):
      if isinstance(node.children[i], tuple): 
        node.create_child(i)
      
      n = node.children[i]

      enc = n.puzzle.encode()

      if enc in s:
        if i == len(node.children) - 1:
          q.pop()
        continue

      s.add(enc)

      if not n.visited:
        q.append(n)
        n.visited = True

        if n.puzzle.check_puzzle():
          return n, create_path(n)

        break

      if i == len(node.children - 1):
        q.pop()

In [65]:
p = Puzzle(min_moves=100, max_moves=150)
t = Tree((-1,-1), p)

p.print_board()

n, path = df_search(t)

[[ 3  1  8]
 [ 2  6 -1]
 [ 5  4  7]]


In [66]:
len(path)

76070

In [67]:
for i, p in enumerate(path):
  p.puzzle.print_board()
  if i == 10: break

[[ 3  1  8]
 [ 2  6 -1]
 [ 5  4  7]]
[[ 3  1 -1]
 [ 2  6  8]
 [ 5  4  7]]
[[ 3 -1  1]
 [ 2  6  8]
 [ 5  4  7]]
[[ 3  6  1]
 [ 2 -1  8]
 [ 5  4  7]]
[[ 3  6  1]
 [ 2  4  8]
 [ 5 -1  7]]
[[ 3  6  1]
 [ 2  4  8]
 [-1  5  7]]
[[ 3  6  1]
 [-1  4  8]
 [ 2  5  7]]
[[-1  6  1]
 [ 3  4  8]
 [ 2  5  7]]
[[ 6 -1  1]
 [ 3  4  8]
 [ 2  5  7]]
[[ 6  4  1]
 [ 3 -1  8]
 [ 2  5  7]]
[[ 6  4  1]
 [ 3  5  8]
 [ 2 -1  7]]


## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento



Busca em Largura: Gasta mais memória, porque precisa gerar todos movimentos, de todos ramos, feitos até a profundidade em que encontrar a melhor solução, mantendo armazenados todos nodos do nivel atual.
    A vantagem é que, como explora todos os movimentos, sempre vai encontrar o melhor caminho.

Busca em Profundidade: Gasta menos memória, já que armazena somente os movimentos executados na busca atual... Porém, como explora cada ramo até não encontrar mais movimentos que gerem estados ainda não encontrados, ele pode seguir caminhos muito longos.

Para garantir que sempre será encontrada alguma solução, é necessário armazenar todos estados já encontrados.

(Para esse problema, com um tabuleiro 3x3, sempre vai existir uma solução curta, porém possui muitos estados que podem ser visitados, fazendo com que normalmente a Busca em Largura, que não precisa procurar muito longe em caminhos demorados, encontre um resultado mais rápido)